# Amazon Titan による抽象的なテキストの要約

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## 概要
大規模な文書を扱う場合、入力テキストがモデルコンテキストの長さに収まらない、モデルが大きなドキュメントによってハルシネーション (文脈と無関係な内容や、事実とは異なる内容が出力される現象) を起こす、メモリ不足エラーなどの問題に直面する可能性があります。

これらの問題を解決するために、プロンプトをチャンク化およびチェーン化するという概念に基づいたアーキテクチャを紹介します。このアーキテクチャは、言語モデルを利用したアプリケーション開発するための一般的なフレームワークである [LangChain](https://python.langchain.com/docs/get_started/introduction.html) を活用しています。

### アーキテクチャ

![](./images/42-text-summarization-2.png)

このアーキテクチャでは:

1. 大きな文書 (または小さなファイルを追加した巨大なファイル) を読み込む
1. Langchain ユーティリティを使用して、小さなチャンクに分割する (チャンク化)
1. 最初のチャンクがモデルに送信され、モデルは対応する要約を返す
2. Langchain は次のチャンクを取得して返された要約に追加し、結合されたテキストを新しいリクエストとしてモデルに送信する
3. 最終的には、コンテンツ全体に基づいた最終的な要約が出来上がる

### ユースケース
このアプローチは、通話記録、会議の記録、書籍、記事、ブログの投稿、およびその他の関連コンテンツを要約するために使用できます。

## セットアップ

このノートブックの残りを実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrock に接続する必要があります。

セットアップの仕組みと ⚠️ **変更が必要か**の詳細については、[Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ja.ipynb) ノートブックを参照してください.

このノートブックでは、入力プロンプトのトークン数をカウントするために利用する [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) ライブラリもインストールします。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249 "transformers>=4.24,<5"

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ コメントを外して、AWS の設定に応じて以下の行を編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## 長いテキストを要約

### Boto3 で LangChain を設定

LangChain に boto3 のセッション情報を渡すと、LangChain から Bedrock にアクセスできるようになります。LangChain に boto3 セッション情報として None を渡すと、LangChain は環境からセッション情報を取得を試みます。 
正しいクライアントが使われるようにするために、ユーティリティメソッドを使ってクライアントをインスタンス化します。

LangChain の Bedrock クラスには LLM を指定する必要があり、推論用の引数を渡すことができます。 ここでは、`model_id` に Amazon Titan Text Large を指定し、`textGenerationConfig` に Titan の推論パラメータを渡します。

In [ ]:
from langchain.llms.bedrock import Bedrock

llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    model_kwargs={
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 1,
    },
    client=boto3_bedrock,
)

### トークンの多いテキストファイルの読み込み

`letters` ディレクトリに、[Amazon の CEO が 2022 年に株主に宛てた手紙](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders)のテキストファイルがあります。以下のセルはテキストファイルを読み込み、ファイル内のトークン数をカウントします。

テキストファイルのトークン数がモデルの最大トークン数を超えていることを示す警告が表示されます。

In [ ]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

### 長いテキストをチャンクに分割

テキストが長すぎてプロンプトに収まらないので、小さなチャンクに分割します。
LangChain の `RecursiveCharacterTextSplitter` は長いテキストをチャンクに分割する機能で、各チャンクのサイズが `chunk_size` より小さくなるまで再起的に分割します。テキストは `separators=["\n\n", "\n"]` で区切られたチャンクに分割されるので、段落後ごとに複数のチャンクに分割されることはありません。

チャンクあたり 6,000 文字を使うことで、各部分の要約を別々に得ることができます。チャンク内のトークン、つまり単語の断片の数はテキストに依存します。

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

### チャンクの要約と結合

トークンの数が他のドキュメントと一致していると仮定すれば問題ないでしょう。LangChain の [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html) を使用して、テキストを要約してみましょう。`load_summarize_chain` は `stuff`、`map_reduce` と `refine` に 3 つの要約方法を提供します。
- `stuff` は全てのチャンクを 1 つのプロンプトにまとめる。そのため、トークンの上限を超えてしまう。
- `map_reduce` は各チャンクを要約し、要約を結合し、結合された要約を要約する。
- `refine` は最初のチャンクを要約し、2 番目のチャンクを最初の要約とともに要約する。全てのチャンクの要約が終わるまで、同じ処理を繰り返す。

`map_reduce` と `refine` は LLM を複数回呼び出すため、最終的な要約を取得するのに時間がかかります。
ここで、`map_reduce` を使ってみましょう。

In [ ]:
# 使用されているプロンプトを確認するには verbose=True を設定してください
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

> ⏰ **注意:** ドキュメントの数、Bedrock のリクエストレートクォータ、再試行設定によってこのチェーンの実行に時間がかかる場合があります。

In [ ]:
output = summary_chain.run(docs)

In [ ]:
print_ww(output.strip())